In [1]:
import xml.dom.minidom
from glob import glob
import html
import unicodedata

import pandas as pd

In [2]:
import dagstermill

In [3]:
debats_xml =  glob('data/debats/xml/compteRendu/*.xml')

len(debats_xml)

310

In [4]:
def getText(nodelist):
    # Iterate all Nodes aggregate TEXT_NODE
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
        else:
            # Recursive
            rc.append(getText(node.childNodes))
    return ''.join(rc)

In [5]:
def get_interventions(debat_xml):
    doc = xml.dom.minidom.parse(debat_xml)
    paragraphes = doc.getElementsByTagName("paragraphe")
    seance = doc.getElementsByTagName('seanceRef')[0].childNodes[0].nodeValue
    interventions = []
    
    for p in paragraphes:
        id_syceron = p.getAttribute('id_syceron')
        acteur = p.getAttribute('id_acteur')
        code_grammaire = p.getAttribute('code_grammaire')
        code_style = p.getAttribute('code_style')
        #texte = p.getElementsByTagName('texte')[0]
        stime = p.getElementsByTagName('texte')[0].getAttribute('stime')
        #texte =  unicodedata.normalize("NFKD", " ".join([ n.nodeValue for n in p.getElementsByTagName('texte')[0].childNodes if n.nodeValue != None ]))
        texte =  unicodedata.normalize("NFKD", getText(p.getElementsByTagName('texte')))

        interventions.append({
            'id_syceron': id_syceron,
            'acteur': acteur,
            'code_grammaire': code_grammaire,
            'code_style': code_style,
            'stime': stime,
            'texte': texte
        })
    
    return (
        pd
        .DataFrame.from_records(interventions)
        .query('acteur != ""')
        .assign(seance = seance)
    )
    #return len(paragraphes)

interventions = pd.concat([get_interventions(debat) for debat in debats_xml])

interventions

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance
0,3043899,PA719874,OUV_SEAN_2_1,NORMAL,986.96,La séance est ouverte.,RUANR5L16S2023IDS26877
2,3043904,PA719874,ODJ_APPEL_DISCUSSION,NORMAL,990.36,L’ordre du jour appelle la discussion du proje...,RUANR5L16S2023IDS26877
3,3043906,PA719874,PAROLE_GENERIQUE,NORMAL,1007.69,La parole est à Mme la ministre de la transit...,RUANR5L16S2023IDS26877
4,3043907,PA759832,PAROLE_GENERIQUE,NORMAL,1041.01,"Après la promulgation, vendredi, de la loi re...",RUANR5L16S2023IDS26877
5,3043912,PA719372,INTERRUPTION_1_10,NORMAL,1087.99,Elle a raison !,RUANR5L16S2023IDS26877
...,...,...,...,...,...,...,...
201,3030326,PA774958,PAROLE_GENERIQUE,NORMAL,16413.59,Je remercie le groupe GDR-NUPES d’avoir mis ce...,RUANR5L16S2023IDS26837
202,3030433,PA720908,PAROLE_GENERIQUE,NORMAL,16528.85,La parole est à M. le ministre.,RUANR5L16S2023IDS26837
203,3030434,PA721498,PAROLE_GENERIQUE,NORMAL,16561.60,Je le dis sous le contrôle de Mme la préside...,RUANR5L16S2023IDS26837
204,3030339,PA720908,PAROLE_GENERIQUE,NORMAL,16779.74,Le débat est clos.,RUANR5L16S2023IDS26837


In [6]:
interventions.query('id_syceron == "2913033"')

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance
664,2913033,PA795778,INTERRUPTION_1_10,NORMAL,,Fake news !,RUANR5L16S2023IDS26502


In [7]:
interventions_extra = (
    interventions
    .assign(
        mots = lambda df: df.texte.apply(lambda texte: len(texte.split(' ')))
    )
)

interventions_extra

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots
0,3043899,PA719874,OUV_SEAN_2_1,NORMAL,986.96,La séance est ouverte.,RUANR5L16S2023IDS26877,4
2,3043904,PA719874,ODJ_APPEL_DISCUSSION,NORMAL,990.36,L’ordre du jour appelle la discussion du proje...,RUANR5L16S2023IDS26877,38
3,3043906,PA719874,PAROLE_GENERIQUE,NORMAL,1007.69,La parole est à Mme la ministre de la transit...,RUANR5L16S2023IDS26877,11
4,3043907,PA759832,PAROLE_GENERIQUE,NORMAL,1041.01,"Après la promulgation, vendredi, de la loi re...",RUANR5L16S2023IDS26877,321
5,3043912,PA719372,INTERRUPTION_1_10,NORMAL,1087.99,Elle a raison !,RUANR5L16S2023IDS26877,4
...,...,...,...,...,...,...,...,...
201,3030326,PA774958,PAROLE_GENERIQUE,NORMAL,16413.59,Je remercie le groupe GDR-NUPES d’avoir mis ce...,RUANR5L16S2023IDS26837,425
202,3030433,PA720908,PAROLE_GENERIQUE,NORMAL,16528.85,La parole est à M. le ministre.,RUANR5L16S2023IDS26837,7
203,3030434,PA721498,PAROLE_GENERIQUE,NORMAL,16561.60,Je le dis sous le contrôle de Mme la préside...,RUANR5L16S2023IDS26837,580
204,3030339,PA720908,PAROLE_GENERIQUE,NORMAL,16779.74,Le débat est clos.,RUANR5L16S2023IDS26837,4


In [8]:
# 9046284
# 9249922
interventions_extra.mots.sum()

9249922

In [9]:
interventions_extra.to_csv('out/interventions.csv', index=False)

In [10]:
dagstermill.yield_result(interventions_extra, output_name="interventions")

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots
0,3043899,PA719874,OUV_SEAN_2_1,NORMAL,986.96,La séance est ouverte.,RUANR5L16S2023IDS26877,4
2,3043904,PA719874,ODJ_APPEL_DISCUSSION,NORMAL,990.36,L’ordre du jour appelle la discussion du proje...,RUANR5L16S2023IDS26877,38
3,3043906,PA719874,PAROLE_GENERIQUE,NORMAL,1007.69,La parole est à Mme la ministre de la transit...,RUANR5L16S2023IDS26877,11
4,3043907,PA759832,PAROLE_GENERIQUE,NORMAL,1041.01,"Après la promulgation, vendredi, de la loi re...",RUANR5L16S2023IDS26877,321
5,3043912,PA719372,INTERRUPTION_1_10,NORMAL,1087.99,Elle a raison !,RUANR5L16S2023IDS26877,4
...,...,...,...,...,...,...,...,...
201,3030326,PA774958,PAROLE_GENERIQUE,NORMAL,16413.59,Je remercie le groupe GDR-NUPES d’avoir mis ce...,RUANR5L16S2023IDS26837,425
202,3030433,PA720908,PAROLE_GENERIQUE,NORMAL,16528.85,La parole est à M. le ministre.,RUANR5L16S2023IDS26837,7
203,3030434,PA721498,PAROLE_GENERIQUE,NORMAL,16561.60,Je le dis sous le contrôle de Mme la préside...,RUANR5L16S2023IDS26837,580
204,3030339,PA720908,PAROLE_GENERIQUE,NORMAL,16779.74,Le débat est clos.,RUANR5L16S2023IDS26837,4
